In [321]:
import pandas as pd
import numpy as np
import os

from tmdbv3api import Movie
from tmdbv3api import TMDb
import requests
import json

In [322]:
data = pd.read_csv("MovieFinal.csv")

In [323]:
data.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,overview,combine
0,Underworld: Blood Wars,Fantasy Action Thriller Horror,Anna Foerster,Kate Beckinsale,Theo James,Lara Pulver,Vampire death dealer Selene fends off brutal a...,Underworld: Blood Wars Fantasy Action Thriller...
1,Arsenal,Thriller Crime,Steven C. Miller,Adrian Grenier,Johnathon Schaech,Nicolas Cage,After the deadbeat brother of a businessman is...,Arsenal Thriller Crime Steven C. Miller Adria...
2,Between Us,Drama,Rafael Palacio Illingworth,Olivia Thirlby,Ben Feldman,Adam Goldberg,A New York couple and their Midwestern friends...,Between Us Drama Rafael Palacio Illingworth O...
3,Monster Trucks,Action Comedy Science Fiction,Chris Wedge,Lucas Till,Jane Levy,Amy Ryan,Tripp is a high school senior with a knack for...,Monster Trucks Action Comedy Science Fiction C...
4,The Bye Bye Man,Horror Thriller,Stacy Title,Douglas Smith,Doug Jones,Michael Trucco,When three college students move into an old h...,The Bye Bye Man Horror Thriller Stacy Title D...


In [324]:
df2 = data[data.duplicated(keep=False)]

df2.sort_values("movie_title").head(10)


,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,overview,combine


In [325]:
data = data.reset_index(drop = True)

In [326]:
len(data)

6730

In [327]:
data["movie_title"][20]

'Youth in Oregon'

In [328]:
data.describe()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,overview,combine
count,6730,6726,6622,6715,6674,6544,6686,6730
unique,6668,1418,3877,3249,4353,4895,6477,6730
top,The Misfits,Drama,Steven Spielberg,Robert De Niro,Bruce Willis,Steve Coogan,"Peter Quill, still reeling from the loss of Ga...",Underworld: Blood Wars Fantasy Action Thriller...
freq,2,396,26,47,18,8,5,1


In [329]:
data.drop_duplicates(subset="movie_title",
                     keep=False, inplace=True)

data.describe()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,overview,combine
count,6606,6602,6503,6591,6550,6425,6563,6606
unique,6606,1400,3823,3214,4282,4821,6398,6606
top,Underworld: Blood Wars,Drama,Steven Spielberg,Robert De Niro,Bruce Willis,Steve Coogan,"Peter Quill, still reeling from the loss of Ga...",Underworld: Blood Wars Fantasy Action Thriller...
freq,1,391,26,47,17,7,5,1


In [330]:
df3 = data[data.duplicated(keep=False)]

df3.sort_values("movie_title").head(10)


,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,overview,combine


In [331]:
def remove_EOL(MovieName):
    return MovieName.split("\xa0")[0]

In [332]:
data["movie_title"] = data["movie_title"].apply(remove_EOL)

In [333]:
data["movie_title"][1773]

'Shazam! Fury of the Gods'

In [334]:
data[data["movie_title"]== "A Cure for Wellness"]

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,overview,combine
28,A Cure for Wellness,Horror Mystery Science Fiction,Gore Verbinski,Dane DeHaan,Mia Goth,Jason Isaacs,An ambitious young executive is sent to retrie...,A Cure for Wellness Horror Mystery Science Fic...


In [339]:
def get_poster(movie_name):
    tmdb = TMDb()
    movie = Movie()
    tmdb.api_key = os.environ.get('api_key')

    url_string = "https://image.tmdb.org/t/p/original"
    result = movie.search(movie_name)
    
    if len(result) > 0 :
        id = result[0]["id"]
    else:
        id = np.NaN

    url3 = f"http://api.themoviedb.org/3/movie/{id}?api_key={tmdb.api_key}"
    response = requests.get(url3)
    data3 = response.json()

    if len(data3) > 3:
          try:
               return url_string+data3.get("poster_path")
          except TypeError:
                return np.NaN     
    

In [340]:
data["poster"] = data["movie_title"].apply(get_poster)

In [346]:
data.to_csv("MovieFinal.csv", index=False)